# **Question Formualation & Data Analysis**

## **Preprocessing**

- Xử lý dữ liệu trùng lặp
- Xử lý giá trị thiếu
- Xử lý dữ liệu không hợp lệ
- Xử lý dữ liệu ngoại lai


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

# Đọc dữ liệu
df = pd.read_csv('../data/raw/student-combine.csv')

In [ ]:
# Xử lý trùng lặp
df = df.drop_duplicates()
df = df.reset_index(drop=True)
# Không có giá trị thiếu => bỏ qua
# Xử lý dữ liệu không hợp lệ
df = df[(df['age'] > 0) & (df['G1'] >= 0) & (df['G2'] >= 0) & (df['G3'] >= 0)]
df = df.reset_index(drop=True)
# Chuyển đổi các cột chỉ yes/no thành 1/0
yes_no_columns = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
for col in yes_no_columns:
    df[col] = df[col].map({'yes': 1, 'no': 0})

# Xử lý outliers bằng winsorization, không dùng thư viện
def winsorize_series(series, lower_percentile=0.01, upper_percentile=0.99):
    lower_bound = series.quantile(lower_percentile)
    upper_bound = series.quantile(upper_percentile)
    return series.clip(lower=lower_bound, upper=upper_bound)

# ap dụng winsorization cho các cột số trừ cột binary
'''
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
binary_columns = yes_no_columns
for col in numeric_columns:
    if col not in binary_columns:
        df[col] = winsorize_series(df[col])
'''
# xử lý điểm G3=0
df = df[df['G3'] > 0].copy()
print(f"Tổng số học sinh tham gia phân tích: {df.shape[0]}")

### **Q1:** Mối liên hệ giữa các yếu tố nhân khẩu học (Giới tính, Tuổi tác) và thói quen học tập (Studytime) ảnh hưởng như thế nào đến Hiệu suất Học tập (Learning Efficiency) trong các môn học?

### **The question**
- **Câu hỏi chính:** Mối liên hệ giữa các yếu tố nhân khẩu học (Giới tính, Tuổi tác) và thói quen học tập (Studytime) ảnh hưởng như thế nào đến Hiệu suất Học tập (Learning Efficiency) trong các môn học?

- **Cụ thể, chúng ta tìm kiếm câu trả lời cho câu hỏi:**
    + Việc tăng thời gian học có tỷ lệ thuận với việc tăng Hiệu suất không, hay tồn tại quy luật 'Hiệu suất giảm dần'?
    + Giới tính nào có chỉ số Hiệu suất chuyển đổi (ROI - Return on Investment) cao hơn? Có sự chênh lệch 'Input cao - Output thấp' ở giới tính nào không?
    + Tuổi tác ảnh hưởng tiêu cực đến Hiệu suất như thế nào? Tại độ tuổi nào (18, 19 hay 20) thì hiệu suất bắt đầu lao dốc không phanh?
    + Các quy luật trên (về Giới tính, Tuổi tác) diễn ra đồng nhất ở cả hai môn hay có sự khác biệt giữa môn Toán (Tư duy Logic) và môn Văn (Tư duy Ngôn ngữ)?
    + Phân tích mối tương tác giữa các yếu tố nhân khẩu học và thói quen học tập ảnh hưởng như thế nào đến Hiệu suất Học tập?

### **Motivation and Benefits**

- Câu hỏi đáng để nghiên cứu:
    + Hiểu rõ hơn về mối quan hệ giữa thời gian học và kết quả học tập để tối ưu hóa chiến lược học tập.
    + Giúp giáo viên và nhà trường thiết kế chương trình giảng dạy phù hợp hơn với từng nhóm học sinh dựa trên đặc điểm nhân khẩu học và thói quen học tập.
    + Cung cấp thông tin hữu ích cho phụ huynh trong việc hỗ trợ con cái trong quá trình học tập.

- Lợi ích và Insight mang lại: 
    + Hiểu rõ hơn về cách các yếu tố nhân khẩu học và thói quen học tập ảnh hưởng đến hiệu suất học tập, từ đó có thể đưa ra các chiến lược cải thiện hiệu quả học tập.
    + Xác định các nhóm học sinh có hiệu suất học tập thấp để có biện pháp hỗ trợ kịp thời.
    + Cung cấp dữ liệu để phát triển các chương trình đào tạo cá nhân hóa dựa trên đặc điểm của từng học sinh.

- Đối tượng quan tâm:

    + Giáo viên và nhà trường: Để cải thiện phương pháp giảng dạy và hỗ trợ học sinh.
    + Phụ huynh: Để hiểu rõ hơn về cách hỗ trợ con cái trong việc học tập.
    + Học sinh: Để nhận thức về thói quen học tập và cách cải thiện hiệu suất học tập của mình.

- Giải quyết vấn đề thực tế:

    + Cung cấp thông tin để thiết kế các chương trình học tập hiệu quả hơn.
    + Giúp học sinh và phụ huynh hiểu rõ hơn về mối quan hệ giữa thói quen học tập và kết quả học tập.
    + Hỗ trợ giáo viên trong việc phát hiện và hỗ trợ những học sinh có hiệu suất học tập thấp.

In [ ]:
# Thêm cột efficiency = G3 / actual_hours để đánh giá hiệu quả học tập
studytime_to_hours = {
    1: 1.5,   # <2h → midpoint = 1.5h
    2: 3.5,   # 2-5h → midpoint = 3.5h  
    3: 7.5,   # 5-10h → midpoint = 7.5h
    4: 12     # >10h → estimate = 12h
}

df['actual_hours'] = df['studytime'].map(studytime_to_hours)
df['efficiency'] = df['G3'] / df['actual_hours']

In [ ]:
# CÂU HỎI 1: HIỆU SUẤT TĂNG THEO THỜI GIAN HỌC KHÔNG?

# 1. Tạo list chứa dữ liệu tổng hợp
summary_data = []

for level in [1, 2, 3, 4]:
    level_data = df[df['studytime'] == level]
    
    # Tính toán các chỉ số
    hours = studytime_to_hours[level]
    count = len(level_data)
    g3_mean = level_data['G3'].mean()
    eff_mean = level_data['efficiency'].mean()
    
    # Lưu vào dictionary
    summary_data.append({
        'Level': level,
        'Giờ học (h/tuần)': hours,
        'Số lượng': count,
        'G3 Trung bình': g3_mean,
        'G3 Range': f"{level_data['G3'].min()} - {level_data['G3'].max()}",
        'Hiệu suất TB': eff_mean,
        'Hiệu suất Range': f"{level_data['efficiency'].min():.2f} - {level_data['efficiency'].max():.2f}"
    })

# 2. Tạo DataFrame
df_summary_q1 = pd.DataFrame(summary_data)

# 3. Tính độ tăng trưởng (Growth Rate) của Hiệu suất
# Dùng hàm diff() để trừ dòng sau cho dòng trước
df_summary_q1['Thay đổi (Điểm/Giờ)'] = df_summary_q1['Hiệu suất TB'].diff()
df_summary_q1['Thay đổi (%)'] = df_summary_q1['Hiệu suất TB'].pct_change() * 100

# 4. Làm đẹp bảng (Fill NaN dòng đầu tiên bằng 0 hoặc -)
df_summary_q1.fillna(0, inplace=True)

# 5. Hiển thị bảng đẹp (Styling)
# Format số liệu để dễ đọc
display(df_summary_q1.style.format({
    'G3 Trung bình': '{:.2f}',
    'Hiệu suất TB': '{:.3f}',
    'Thay đổi (Điểm/Giờ)': '{:+.3f}', # Thêm dấu +/- đằng trước
    'Thay đổi (%)': '{:+.1f}%'        # Thêm dấu %
}))

# Visualize - Đa dạng hóa các loại biểu đồ
fig, axes = plt.subplots(3, 1, figsize=(14, 20))

# Biểu đồ 1: Bar chart - G3 theo studytime
level_labels = ['<2h', '2-5h', '5-10h', '>10h']
g3_means = [df[df['studytime'] == i]['G3'].mean() for i in [1,2,3,4]]
colors_g3 = ['#FF6B6B', '#FFA726', '#66BB6A', '#42A5F5']

axes[0].bar(level_labels, g3_means, color=colors_g3, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Điểm G3 trung bình', fontsize=11, fontweight='bold')
axes[0].set_title('Điểm G3 theo mức độ studytime (Bar Chart)', fontsize=13, fontweight='bold', pad=12)
axes[0].set_ylim([0, max(g3_means) * 1.15])
axes[0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(g3_means):
    axes[0].text(i, v + 0.3, f'{v:.2f}', ha='center', fontweight='bold', fontsize=10)


# Biểu đồ 2: Box plot - So sánh hiệu suất theo studytime
box_data = [df[df['studytime'] == i]['efficiency'].values for i in [1,2,3,4]]
bp = axes[1].boxplot(box_data, labels=level_labels, patch_artist=True)
for patch, color in zip(bp['boxes'], colors_g3):
    patch.set_facecolor(color)
axes[1].set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
axes[1].set_title('Hiệu suất học tập - So sánh các mức độ (Box Plot)', fontsize=13, fontweight='bold', pad=12)
axes[1].set_ylim([0, max([max(bd) for bd in box_data]) * 1.1])
axes[1].grid(True, alpha=0.3, axis='y')

# Biểu đồ 3: Bar chart gradient - Xu hướng hiệu suất rõ ràng
eff_means = [df[df['studytime'] == i]['efficiency'].mean() for i in [1,2,3,4]]

# Normalize giá trị để màu gradient
eff_norm = [(e - min(eff_means)) / (max(eff_means) - min(eff_means)) for e in eff_means]
bar_colors = colors_g3

bars = axes[2].bar(range(len(level_labels)), eff_means, color=bar_colors, 
                    edgecolor='black', linewidth=2.5, alpha=0.9, width=0.65)

# Vẽ một đường trend để chỉ độ dốc giảm
axes[2].plot(range(len(level_labels)), eff_means, color='#C0392B', linewidth=3.5, 
             marker='o', markersize=12, alpha=0.8, markeredgecolor='black', 
             markeredgewidth=2, linestyle='-', label='Xu hướng giảm dần')

# Thêm labels cho mỗi bar
for i, (bar, val) in enumerate(zip(bars, eff_means)):
    axes[2].text(i, val + 0.3, f'{val:.3f}', ha='center', fontweight='bold', fontsize=11,
                bbox=dict(boxstyle='round,pad=0.4', facecolor='white', edgecolor='black', linewidth=1))

axes[2].set_xlabel('Mức độ Studytime', fontsize=11, fontweight='bold')
axes[2].set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
axes[2].set_title('Biểu đồ 3: Hiệu Suất Giảm Dần theo Thời gian Học (Bar + Trend Line)', fontsize=13, fontweight='bold', pad=12)
axes[2].set_xticks(range(len(level_labels)))
axes[2].set_xticklabels(level_labels, fontsize=11)
axes[2].set_ylim([0, max(eff_means) * 1.4])
axes[2].legend(fontsize=11, loc='upper right')
axes[2].grid(True, alpha=0.3, axis='y', linestyle='--')

plt.tight_layout()


## NHẬN ĐỊNH CÂU HỎI 1: Liệu thời gian học có tỷ lệ thuận với hiệu suất?

**PHÁT HIỆN CHÍNH:**

Điểm G3 tăng từ **11.33** (Level 1) lên **12.97** (Level 3), nhưng hiệu suất lại giảm mạnh từ **7.554** xuống **1.075** điểm/giờ (giảm 85.8%). Biểu đồ 3 cho thấy đường cong giảm dần rõ ràng, chứng minh **Law of Diminishing Returns**.

**MÂU THUẪN:**
- Level 4 học nhiều gấp 8 lần Level 1 (12h vs 1.5h) nhưng hiệu suất chỉ bằng 1/7
- Từ Level 3→4: Học thêm 4.5h/tuần nhưng điểm gần như không tăng (-0.07)

**DỰ ĐOÁN NGUYÊN NHÂN:**
Burnout tâm lý, chất lượng học tập giảm, não bộ bão hòa thông tin, và áp lực học tập tăng cao.

**KẾT LUẬN:**
**Mức tối ưu: Level 3 (5-10h/tuần)** - cân bằng giữa điểm cao và hiệu suất hợp lý. Học quá nhiều không mang lại lợi ích tương xứng.

In [ ]:
# CÂU HỎI 2: GIỚI TÍNH NÀO CÓ HIỆU SUẤT CHUYỂN ĐỔI (ROI) CAO HƠN?

# ---------------------------------------------------------
# PHẦN 1: BẢNG TỔNG QUAN THEO GIỚI TÍNH
# ---------------------------------------------------------
gender_stats = []
for sex in ['F', 'M']:
    sex_data = df[df['sex'] == sex]
    sex_name = 'Nữ' if sex == 'F' else 'Nam'
    
    gender_stats.append({
        'Giới tính': sex_name,
        'Số lượng': len(sex_data),
        'G3 TB': sex_data['G3'].mean(),
        'Min-Max G3': f"{sex_data['G3'].min()} - {sex_data['G3'].max()}",
        'Giờ học TB': sex_data['actual_hours'].mean(),
        'Hiệu suất TB': sex_data['efficiency'].mean()
    })

df_gender_overview = pd.DataFrame(gender_stats)

print("1. THỐNG KÊ HIỆU SUẤT TỔNG QUAN:")
# Hiển thị bảng đẹp với gradient màu xanh
display(df_gender_overview.style.format({
    'G3 TB': '{:.2f}',
    'Giờ học TB': '{:.2f}',
    'Hiệu suất TB': '{:.3f}'
}).background_gradient(subset=['Hiệu suất TB'], cmap='Blues'))


# ---------------------------------------------------------
# PHẦN 2: BẢNG CHI TIẾT THEO MỨC ĐỘ STUDYTIME
# ---------------------------------------------------------
gender_level_stats = []
# Pre-calculate grouped means for speed
grouped = df.groupby(['sex', 'studytime'])[['G3', 'efficiency']].mean()

for sex in ['F', 'M']:
    sex_name = 'Nữ' if sex == 'F' else 'Nam'
    for level in [1, 2, 3, 4]:
        try:
            g3 = grouped.loc[(sex, level), 'G3']
            eff = grouped.loc[(sex, level), 'efficiency']
            hours = studytime_to_hours[level]
            
            gender_level_stats.append({
                'Giới tính': sex_name,
                'Mức học': f'Level {level}',
                'Giờ quy đổi': hours,
                'G3 TB': g3,
                'Hiệu suất TB': eff
            })
        except KeyError:
            pass # Bỏ qua nếu không có dữ liệu

df_gender_detail = pd.DataFrame(gender_level_stats)

print("\n2. CHI TIẾT HIỆU SUẤT THEO TỪNG MỨC ĐỘ HỌC:")
# Hiển thị bảng đẹp, tô màu để so sánh hiệu suất
display(df_gender_detail.style.format({
    'G3 TB': '{:.2f}',
    'Hiệu suất TB': '{:.3f}'
}).background_gradient(subset=['Hiệu suất TB'], cmap='RdYlGn'))


# ---------------------------------------------------------
# PHẦN 3: VISUALIZATION (GIỮ NGUYÊN)
# ---------------------------------------------------------
fig, axes = plt.subplots(3, 1, figsize=(18, 24))
fig.suptitle('Câu Hỏi 2: Hiệu Suất Học Tập - So Sánh Nam-Nữ', fontsize=18, fontweight='bold', y=1.02)

# Biểu đồ 1: Boxplot
bp_data = [df[df['sex'] == sex]['efficiency'].values for sex in ['F', 'M']]
bp = axes[0].boxplot(bp_data, labels=['Nữ', 'Nam'], patch_artist=True, widths=0.5, showmeans=True,
                      meanprops=dict(marker='D', markerfacecolor='red', markersize=8))
for patch, color in zip(bp['boxes'], ['#FF6B9D', '#4A90E2']):
    patch.set_facecolor(color)
    patch.set_alpha(0.75)
    patch.set_linewidth(1.5)
axes[0].set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
axes[0].set_title('Biểu đồ 1: Phân bố Hiệu suất theo Giới Tính', fontsize=12, fontweight='bold', pad=12)
axes[0].grid(True, alpha=0.3, axis='y', linestyle='--')

# Biểu đồ 2: Bar chart
female_eff_mean = df[df['sex'] == 'F']['efficiency'].mean()
male_eff_mean = df[df['sex'] == 'M']['efficiency'].mean()
bars = axes[1].bar(['Nữ', 'Nam'], [female_eff_mean, male_eff_mean], color=['#FF6B9D', '#4A90E2'], 
                    edgecolor='black', linewidth=2, width=0.5, alpha=0.85)
axes[1].set_ylabel('Hiệu suất trung bình (điểm/giờ)', fontsize=11, fontweight='bold')
axes[1].set_title('Biểu đồ 2: So Sánh Hiệu Suất Trung Bình', fontsize=12, fontweight='bold', pad=12)
axes[1].grid(True, alpha=0.3, axis='y', linestyle='--')
axes[1].set_ylim([0, max([female_eff_mean, male_eff_mean]) + 0.15])
for i, (bar, v) in enumerate(zip(bars, [female_eff_mean, male_eff_mean])):
    axes[1].text(i, v + 0.03, f'{v:.3f}', ha='center', fontweight='bold', fontsize=11, color='black')

# Biểu đồ 3: Line + Scatter
female_eff = [df[(df['sex']=='F') & (df['studytime']==i)]['efficiency'].mean() for i in [1,2,3,4]]
male_eff = [df[(df['sex']=='M') & (df['studytime']==i)]['efficiency'].mean() for i in [1,2,3,4]]
level_labels = ['<2h', '2-5h', '5-10h', '>10h']
axes[2].scatter([1, 2, 3, 4], male_eff, s=180, alpha=0.8, color='#4A90E2', label='Nam', 
                marker='s', edgecolors='black', linewidth=1.5, zorder=3)
axes[2].scatter([1, 2, 3, 4], female_eff, s=180, alpha=0.8, color='#FF6B9D', label='Nữ', 
                marker='o', edgecolors='black', linewidth=1.5, zorder=3)
axes[2].plot([1, 2, 3, 4], male_eff, color='#4A90E2', linewidth=2.5, alpha=0.6, linestyle='--')
axes[2].plot([1, 2, 3, 4], female_eff, color='#FF6B9D', linewidth=2.5, alpha=0.6, linestyle='--')
axes[2].set_xlabel('Mức độ studytime', fontsize=11, fontweight='bold')
axes[2].set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
axes[2].set_title('Biểu đồ 3: Hiệu Suất Nam-Nữ theo Studytime', fontsize=12, fontweight='bold', pad=12)
axes[2].set_xticks([1, 2, 3, 4])
axes[2].set_xticklabels(level_labels)
axes[2].legend(fontsize=10, loc='best')
axes[2].grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

## NHẬN ĐỊNH CÂU HỎI 2: Giới tính nào có ROI cao hơn? Có chênh lệch Input-Output?

**PHÁT HIỆN CHÍNH:**

**NAM có ROI cao hơn:** Hiệu suất trung bình Nam = **4.921 điểm/giờ** vs Nữ = **3.712 điểm/giờ** (cao hơn 32.6%). Nam đạt điểm tương đương Nữ nhưng **chỉ cần 3.43h/tuần** so với 4.51h/tuần của Nữ.

**CHÊNH LỆCH INPUT-OUTPUT:**
- **Nam**: Input thấp (3.43h/tuần) → Output trung bình (G3=11.80) → **ROI cao**
- **Nữ**: Input cao (4.51h/tuần) → Output cao hơn (G3=12.06) → **ROI thấp hơn**
- Biểu đồ 3 cho thấy: Ở Level 3, Nam đạt G3=13.69 (cao nhất), Nữ chỉ 12.76. Ở Level 4, Nữ học nhiều nhưng hiệu suất giảm mạnh

**NGHỊCH LÝ LEVEL 4:**
Nữ học 12h/tuần đạt G3=13.47 (eff=1.123), trong khi Nam học cùng mức chỉ đạt G3=12.00 (eff=1.000). Nữ có xu hướng duy trì kết quả ở mức học cao, Nam sụt giảm rõ rệt.

**DỰ ĐOÁN NGUYÊN NHÂN:**
Nam có phương pháp học tập hiệu quả hơn ở mức độ vừa phải, nhưng kém kiên trì ở mức học cao. Nữ học chăm chỉ hơn nhưng hiệu suất từng giờ thấp hơn.

**KẾT LUẬN:**
Nam có **ROI tốt hơn** nhìn chung, nhưng Nữ có **khả năng duy trì** tốt hơn ở mức học cao. Cần chiến lược: Khuyến khích Nam học đều đặn, hỗ trợ Nữ tối ưu phương pháp học.

In [ ]:
# CÂU HỎI 3: TUỔI TÁC ẢNH HƯỞNG ĐẾN HIỆU SUẤT NHƯ THẾ NÀO?

# ---------------------------------------------------------
# PHẦN 1: BẢNG THỐNG KÊ HIỆU SUẤT THEO ĐỘ TUỔI
# ---------------------------------------------------------
age_stats = []
all_ages = sorted(df['age'].unique())

for age in all_ages:
    age_data = df[df['age'] == age]
    count = len(age_data)
    
    # Chỉ đưa vào bảng nếu số lượng mẫu đủ lớn (>5) để đảm bảo độ tin cậy
    if count > 5:
        age_stats.append({
            'Tuổi': age,
            'Số lượng (n)': count,
            'G3 TB': age_data['G3'].mean(),
            'Hiệu suất TB': age_data['efficiency'].mean()
        })

df_age_stats = pd.DataFrame(age_stats)

print("1. THỐNG KÊ HIỆU SUẤT THEO ĐỘ TUỔI:")
display(df_age_stats.style.format({
    'G3 TB': '{:.2f}',
    'Hiệu suất TB': '{:.3f}'
}).background_gradient(subset=['Hiệu suất TB'], cmap='RdYlGn'))


# ---------------------------------------------------------
# PHẦN 2: PHÂN TÍCH ĐIỂM GÃY (SỰ THAY ĐỔI GIỮA CÁC ĐỘ TUỔI)
# ---------------------------------------------------------
change_stats = []

for i in range(len(all_ages) - 1):
    age1, age2 = all_ages[i], all_ages[i+1]
    
    # Tính trung bình hiệu suất
    eff1 = df[df['age'] == age1]['efficiency'].mean()
    eff2 = df[df['age'] == age2]['efficiency'].mean()
    change = eff2 - eff1
    
    # Chỉ ghi nhận những thay đổi đáng kể (> 0.02)
    if abs(change) > 0.02:
        trend = "▼ Suy giảm" if change < 0 else "▲ Tăng"
        color_trend = 'red' if change < 0 else 'green'
        
        change_stats.append({
            'Giai đoạn': f"{age1} ⮕ {age2}",
            'Hiệu suất cũ': eff1,
            'Hiệu suất mới': eff2,
            'Thay đổi': change,
            'Xu hướng': trend
        })

df_age_change = pd.DataFrame(change_stats)

print("\n2. PHÂN TÍCH ĐIỂM GÃY (GIAI ĐOẠN BIẾN ĐỘNG MẠNH):")
# Tạo hàm tô màu chữ cho cột Xu hướng
def color_trend(val):
    color = 'red' if 'Suy giảm' in val else 'green'
    return f'color: {color}; font-weight: bold'

display(df_age_change.style.format({
    'Hiệu suất cũ': '{:.3f}',
    'Hiệu suất mới': '{:.3f}',
    'Thay đổi': '{:+.3f}'
}).applymap(color_trend, subset=['Xu hướng'])
  .bar(subset=['Thay đổi'], align='mid', color=['#FF6B6B', '#66BB6A']))


# ---------------------------------------------------------
# PHẦN 3: VISUALIZATION (GIỮ NGUYÊN)
# ---------------------------------------------------------
fig, axes = plt.subplots(3, 1, figsize=(16, 12))
fig.suptitle('Câu Hỏi 3: Ảnh Hưởng Tuổi Tác Đến Hiệu Suất', fontsize=18, fontweight='bold', y=0.995)

ages = sorted(df['age'].unique())

# ===============================
# 🎨 BIỂU ĐỒ 1 – BAR CHART (GRADIENT VIRIDIS)
# ===============================
colors_age = plt.cm.viridis(np.linspace(0.2, 0.9, len(ages)))

age_g3_mean = [df[df['age'] == age]['G3'].mean() for age in ages]

bars = axes[0].bar(ages, age_g3_mean, color=colors_age, edgecolor='black',
                   linewidth=1.2, width=0.7, alpha=0.9)

axes[0].set_xlabel('Tuổi', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Điểm G3 trung bình', fontsize=11, fontweight='bold')
axes[0].set_title('Biểu đồ 1: Điểm G3 theo Tuổi (Gradient Modern – Viridis)',
                  fontsize=12, fontweight='bold', pad=12)
axes[0].grid(True, alpha=0.3, axis='y', linestyle='--')

for age, v in zip(ages, age_g3_mean):
    axes[0].text(age, v + 0.25, f'{v:.1f}', ha='center',
                 fontweight='bold', fontsize=9)

# ===============================
# 🎨 BIỂU ĐỒ 2 – LINE + SCATTER (GRADIENT RdYlGn_r)
# ===============================
age_eff_mean = [df[df['age'] == age]['efficiency'].mean() for age in ages]

# Đường line xu hướng màu đỏ đậm
axes[1].plot(ages, age_eff_mean, color="#C0392B", linewidth=3.5, alpha=0.8, zorder=2)

# Scatter với gradient xanh (tốt) → đỏ (kém)
colors_gradient = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(ages)))
axes[1].scatter(ages, age_eff_mean, s=250, c=colors_gradient,
                edgecolors='black', linewidth=2.5, alpha=0.9, zorder=3)

# Annotation điểm gãy 17 → 18 (chỉ rõ ràng hơn)
turning_idx = ages.index(17) if 17 in ages else -1
if turning_idx >= 0 and turning_idx < len(ages) - 1:
    # Tính toạ độ chính xác của 2 điểm
    x1, y1 = 17, age_eff_mean[turning_idx]
    x2, y2 = 18, age_eff_mean[turning_idx + 1]
    
    # Annotation chỉ vào điểm giữa 17 và 18
    axes[1].annotate(
        'Điểm gãy\n17→18',
        xy=(17.5, (y1 + y2) / 2),
        xytext=(16, y1 + 0.8),
        fontsize=10, fontweight='bold', ha='center',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='#F39C12', alpha=0.9, edgecolor='#E67E22', linewidth=2),
        arrowprops=dict(arrowstyle='->', color='#C0392B', lw=2.5, connectionstyle='arc3,rad=0.3')
    )

axes[1].set_xlabel('Tuổi', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
axes[1].set_title('Biểu đồ 2: Xu Hướng Hiệu Suất theo Tuổi (Giảm Dần Rõ Rệt)',
                  fontsize=12, fontweight='bold', pad=12)
axes[1].grid(True, alpha=0.3, linestyle='--')

# ===============================
# 🎨 BIỂU ĐỒ 3 – BOX PLOT (GRADIENT MAGMA)
# ===============================
age_groups_data = [(15, 17, '15-17'), (18, 19, '18-19'), (20, 25, '20+')]

box_data, box_labels = [], []
for min_age, max_age, label in age_groups_data:
    group_data = df[(df['age'] >= min_age) & (df['age'] <= max_age)]['G3']
    if len(group_data) > 0:
        box_data.append(group_data)
        box_labels.append(label)

bp = axes[2].boxplot(
    box_data, labels=box_labels, patch_artist=True, widths=0.5,
    showmeans=True, meanprops=dict(marker='D', markerfacecolor="#F39C12", markersize=9)
)

# Box màu theo gradient magma
box_colors = plt.cm.magma(np.linspace(0.3, 0.8, len(box_data)))

for patch, color in zip(bp['boxes'], box_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.92)
    patch.set_linewidth(0)

for whisker in bp['whiskers']:
    whisker.set(color='#555', linewidth=1.8, alpha=0.6)

for median in bp['medians']:
    median.set(color="#000", linewidth=3)

axes[2].set_ylabel('Điểm G3', fontsize=11, fontweight='bold')
axes[2].set_title('Biểu đồ 3: Phân Bố Điểm theo Nhóm Tuổi (Gradient Magma)',
                  fontsize=12, fontweight='bold', pad=12)
axes[2].grid(True, alpha=0.3, axis='y', linestyle='--')

plt.tight_layout()
plt.show()

## NHẬN ĐỊNH CÂU HỎI 3: Tuổi tác ảnh hưởng tiêu cực thế nào? Độ tuổi nào bắt đầu giảm sút?

**PHÁT HIỆN CHÍNH:**

Hiệu suất giảm dần từ **~4.5 điểm/giờ (tuổi 15-16)** xuống **~3.5 điểm/giờ (tuổi 19)**. Điểm G3 ổn định **12.0-12.1** ở tuổi 15-18, sau đó **giảm xuống 10.5** ở tuổi 19 (thấp nhất). Tuổi 20-22 có **biến động bất thường** do sample size quá nhỏ (n=9, 5, 2, 2).

**ĐIỂM GÃY 17→18:** 
Đây là **thời điểm quan trọng** khi hiệu suất bắt đầu suy giảm rõ rệt (được đánh dấu trên Biểu đồ 2). Từ đó tiếp tục giảm ở tuổi 19.

**BOXPLOT 3 NHÓM:**
- **15-17**: Median ~12, phân bố tập trung
- **18-19**: Median ~11, phân bố rộng hơn  
- **20+**: Median ~10, nhiều outlier

**DỰ ĐOÁN NGUYÊN NHÂN:**

Học sinh lớn tuổi thường do **lưu ban** → kiến thức yếu, áp lực tâm lý, mất động lực.

**KẾT LUẬN:**
**Tuổi 18 là điểm gãy** - cần can thiệp sớm ở độ tuổi 17-18 để ngăn suy giảm. Dữ liệu từ tuổi 20+ không tin cậy do sample nhỏ.

In [ ]:
# CÂU HỎI 4: CÁC QUY LUẬT CÓ ĐỒNG NHẤT GIỮA HAI MÔN KHÔNG? (BẢN DATAFRAME)

# ---------------------------------------------------------
# PHẦN 1: SO SÁNH TÁC ĐỘNG CỦA THỜI GIAN HỌC (STUDYTIME)
# ---------------------------------------------------------
studytime_comparison = []

for subject in ['mat', 'por']:
    subj_name = 'Toán' if subject == 'mat' else 'Văn'
    for level in [1, 2, 3, 4]:
        subset = df[(df['subject'] == subject) & (df['studytime'] == level)]
        if not subset.empty:
            hours = studytime_to_hours[level]
            studytime_comparison.append({
                'Môn': subj_name,
                'Mức độ': level,
                'Giờ học (h)': hours,
                'G3 TB': subset['G3'].mean(),
                'Hiệu suất TB': subset['efficiency'].mean()
            })

df_compare_study = pd.DataFrame(studytime_comparison)
# Pivot để so sánh Toán vs Văn cạnh nhau
df_compare_study_pivot = df_compare_study.pivot(index=['Mức độ', 'Giờ học (h)'], columns='Môn', values=['G3 TB', 'Hiệu suất TB'])

print("1. SO SÁNH TÁC ĐỘNG CỦA THỜI GIAN HỌC:")
# Tô màu xanh để thấy hiệu suất giảm dần
display(df_compare_study_pivot.style.format('{:.2f}').background_gradient(cmap='Blues', subset=[('Hiệu suất TB', 'Toán'), ('Hiệu suất TB', 'Văn')]))


# ---------------------------------------------------------
# PHẦN 2: SO SÁNH TÁC ĐỘNG CỦA GIỚI TÍNH (ROI)
# ---------------------------------------------------------
gender_comparison = []

for subject in ['mat', 'por']:
    subj_name = 'Toán' if subject == 'mat' else 'Văn'
    for sex in ['F', 'M']:
        sex_name = 'Nữ' if sex == 'F' else 'Nam'
        subset = df[(df['subject'] == subject) & (df['sex'] == sex)]
        
        gender_comparison.append({
            'Môn': subj_name,
            'Giới tính': sex_name,
            'G3 TB': subset['G3'].mean(),
            'Hiệu suất TB': subset['efficiency'].mean()
        })

df_compare_gender = pd.DataFrame(gender_comparison)
df_compare_gender_pivot = df_compare_gender.pivot(index='Giới tính', columns='Môn', values=['G3 TB', 'Hiệu suất TB'])

print("\n2. SO SÁNH TÁC ĐỘNG CỦA GIỚI TÍNH:")
# Highlight giá trị cao nhất trong từng hàng để xem ai thắng
display(df_compare_gender_pivot.style.format('{:.2f}').highlight_max(axis=1, color='green'))


# ---------------------------------------------------------
# PHẦN 3: SO SÁNH TÁC ĐỘNG CỦA TUỔI TÁC (NHÓM TUỔI)
# ---------------------------------------------------------
age_comparison = []
age_groups = [(15, 17, '15-17'), (18, 19, '18-19'), (20, 25, '20+')]

for subject in ['mat', 'por']:
    subj_name = 'Toán' if subject == 'mat' else 'Văn'
    for min_age, max_age, label in age_groups:
        subset = df[(df['subject'] == subject) & (df['age'] >= min_age) & (df['age'] <= max_age)]
        
        if not subset.empty:
            age_comparison.append({
                'Môn': subj_name,
                'Nhóm tuổi': label,
                'G3 TB': subset['G3'].mean(),
                'Hiệu suất TB': subset['efficiency'].mean()
            })

df_compare_age = pd.DataFrame(age_comparison)
df_compare_age_pivot = df_compare_age.pivot(index='Nhóm tuổi', columns='Môn', values=['G3 TB', 'Hiệu suất TB'])

print("\n3. SO SÁNH TÁC ĐỘNG CỦA TUỔI TÁC:")
# Tô màu đỏ để cảnh báo sự suy giảm hiệu suất
display(df_compare_age_pivot.style.format('{:.2f}').background_gradient(cmap='Reds', subset=[('Hiệu suất TB', 'Toán'), ('Hiệu suất TB', 'Văn')]))


# ---------------------------------------------------------
# PHẦN 4: VISUALIZATION (GIỮ NGUYÊN CODE CŨ CỦA BẠN)
# ---------------------------------------------------------
fig = plt.figure(figsize=(18, 14))
fig.suptitle('Câu Hỏi 4: So Sánh Môn Toán vs Tiếng Bồ\n(Quy luật có đồng nhất giữa hai môn?)', 
             fontsize=18, fontweight='bold')
gs = fig.add_gridspec(3, 2, hspace=0.35, wspace=0.35)

# ========== BIỂU ĐỒ 1: Grouped bar chart - G3 theo studytime ==========
mat_g3 = [df[(df['subject']=='mat') & (df['studytime']==i)]['G3'].mean() for i in [1,2,3,4]]
por_g3 = [df[(df['subject']=='por') & (df['studytime']==i)]['G3'].mean() for i in [1,2,3,4]]

level_labels = ['<2h', '2-5h', '5-10h', '>10h']
x = np.arange(len(level_labels))
width = 0.35

ax1 = fig.add_subplot(gs[0, 0])
bars1 = ax1.bar(x - width/2, mat_g3, width, label='Toán', color='#E74C3C', edgecolor='black', linewidth=1.5, alpha=0.85)
bars2 = ax1.bar(x + width/2, por_g3, width, label='Văn', color='#3498DB', edgecolor='black', linewidth=1.5, alpha=0.85)
ax1.set_ylabel('Điểm G3', fontsize=11, fontweight='bold')
ax1.set_title('Biểu đồ 1: Điểm G3 theo Studytime - Toán vs Văn', fontsize=12, fontweight='bold', pad=12)
ax1.set_xticks(x)
ax1.set_xticklabels(level_labels)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='y', linestyle='--')
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.2,
                f'{height:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# ========== BIỂU ĐỒ 2: Grouped bar chart - Hiệu suất theo giới tính ==========
mat_f_eff = df[(df['subject']=='mat') & (df['sex']=='F')]['efficiency'].mean()
mat_m_eff = df[(df['subject']=='mat') & (df['sex']=='M')]['efficiency'].mean()
por_f_eff = df[(df['subject']=='por') & (df['sex']=='F')]['efficiency'].mean()
por_m_eff = df[(df['subject']=='por') & (df['sex']=='M')]['efficiency'].mean()

ax2 = fig.add_subplot(gs[0, 1])
gender_x = np.arange(2)
bars1 = ax2.bar(gender_x - width/2, [mat_f_eff, mat_m_eff], width, label='Toán', 
                color='#E74C3C', edgecolor='black', linewidth=1.5, alpha=0.85)
bars2 = ax2.bar(gender_x + width/2, [por_f_eff, por_m_eff], width, label='Văn', 
                color='#3498DB', edgecolor='black', linewidth=1.5, alpha=0.85)
ax2.set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
ax2.set_title('Biểu đồ 2: Hiệu Suất theo Giới Tính - Toán vs Văn\n(Quy luật khác nhau theo Nam-Nữ?)', 
              fontsize=12, fontweight='bold', pad=12)
ax2.set_xticks(gender_x)
ax2.set_xticklabels(['Nữ', 'Nam'])
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y', linestyle='--')
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{height:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# ========== BIỂU ĐỒ 3: Line chart - Hiệu suất theo studytime ==========
mat_eff = [df[(df['subject']=='mat') & (df['studytime']==i)]['efficiency'].mean() for i in [1,2,3,4]]
por_eff = [df[(df['subject']=='por') & (df['studytime']==i)]['efficiency'].mean() for i in [1,2,3,4]]

ax3 = fig.add_subplot(gs[1,:])
ax3.plot(x, mat_eff, marker='o', linewidth=3.5, markersize=12, label='Toán', 
         color='#E74C3C', markeredgecolor='black', markeredgewidth=2, zorder=3)
ax3.plot(x, por_eff, marker='s', linewidth=3.5, markersize=12, label='Văn', 
         color='#3498DB', markeredgecolor='black', markeredgewidth=2, zorder=3)
ax3.fill_between(x, mat_eff, alpha=0.2, color='#E74C3C')
ax3.fill_between(x, por_eff, alpha=0.2, color='#3498DB')
ax3.set_xlabel('Mức độ Studytime', fontsize=11, fontweight='bold')
ax3.set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
ax3.set_title('Biểu đồ 3: Hiệu Suất theo Studytime - Toán vs Văn\n(Quy luật giảm dần khác nhau?)', 
              fontsize=12, fontweight='bold', pad=12)
ax3.set_xticks(x)
ax3.set_xticklabels(level_labels)
ax3.legend(fontsize=11, loc='best')
ax3.grid(True, alpha=0.3, linestyle='--')

# ========== BIỂU ĐỒ 4: Line chart - Hiệu suất theo tuổi ==========
ax4 = fig.add_subplot(gs[2, :])
ages = sorted(df['age'].unique())
mat_eff_age = [df[(df['subject']=='mat') & (df['age']==age)]['efficiency'].mean() for age in ages]
por_eff_age = [df[(df['subject']=='por') & (df['age']==age)]['efficiency'].mean() for age in ages]

ax4.plot(ages, mat_eff_age, marker='o', linewidth=3.5, markersize=11, label='Toán', 
         color='#E74C3C', markeredgecolor='black', markeredgewidth=2, zorder=3)
ax4.plot(ages, por_eff_age, marker='s', linewidth=3.5, markersize=11, label='Văn', 
         color='#3498DB', markeredgecolor='black', markeredgewidth=2, zorder=3)
ax4.fill_between(ages, mat_eff_age, alpha=0.15, color='#E74C3C')
ax4.fill_between(ages, por_eff_age, alpha=0.15, color='#3498DB')
ax4.set_xlabel('Tuổi', fontsize=11, fontweight='bold')
ax4.set_ylabel('Hiệu suất (điểm/giờ)', fontsize=11, fontweight='bold')
ax4.set_title('Biểu đồ 4: Hiệu Suất theo Tuổi - Toán vs Văn\n(Quy luật suy giảm có giống nhau?)', 
              fontsize=12, fontweight='bold', pad=12)
ax4.legend(fontsize=11, loc='best')
ax4.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

## NHẬN ĐỊNH CÂU HỎI 4: Các quy luật có đồng nhất giữa hai môn?

**PHÁT HIỆN CHÍNH:** Quy luật không đồng nhất; Toán (logic-based) và Văn (ngôn ngữ) phản ứng khác nhau với thời gian học, giới tính và tuổi tác.

**Studytime:**
- Toán: Điểm G3 tăng đều từ <2h đến >10h; hiệu suất giảm nhưng dốc nhẹ hơn, duy trì ~2→1 điểm/giờ.
- Văn: Điểm G3 phẳng hơn; hiệu suất giảm mạnh từ ~7 xuống ~1 khi tăng giờ học → hiệu suất bão hòa rõ.

**Giới tính:**
- Toán: Nam ~5.0, Nữ ~3.2 điểm/giờ → Nam ROI cao hơn.
- Văn: Nam ~5.0, Nữ ~3.6 điểm/giờ nhưng cả hai đều cao hơn Toán → lợi thế giới tính ở Văn ít chênh lệch hơn.

**Tuổi tác:**
- Toán: Xu hướng xuống dốc sau 18; dao động mạnh hơn, dễ thấy suy giảm ở 20-22.
- Văn: Xu hướng bằng phẳng hơn đến 19 rồi giảm nhẹ; ít biến động hơn ở nhóm 20+ (dù sample nhỏ).

**KẾT LUẬN:** Không thể dùng một quy luật chung. Toán cần đầu tư giờ học nhưng hiệu suất giảm dần; Văn đạt điểm ổn với ít giờ hơn nhưng cũng bão hòa sớm. Nam lợi thế rõ ở Toán, còn ở Văn chênh lệch giới nhỏ hơn. **Cần can thiệp khẩn cấp ở tuổi 17-19** (nhóm lưu ban) để ngăn suy giảm tiếp tục, và điều chỉnh chiến lược theo từng môn.

In [ ]:
# PHÂN TÍCH MỐI LIÊN HỆ TƯƠNG TÁC (DATAFRAME VERSION)

print("="*80)
print("PHÂN TÍCH MỐI LIÊN HỆ TƯƠNG TÁC GIỮA CÁC YẾU TỐ")
print("="*80)

# Định nghĩa lại để đảm bảo code chạy độc lập
studytime_to_hours = {1: 1.5, 2: 3.5, 3: 7.5, 4: 12}

# ==============================================================================
# 1️⃣ TƯƠNG TÁC: STUDYTIME × GENDER (DATAFRAME)
# ==============================================================================
interaction_gender_data = []

for subject in ['mat', 'por']:
    subject_name = 'Toán' if subject == 'mat' else 'Văn'
    for level in [1, 2, 3, 4]:
        hours = studytime_to_hours[level]
        
        # Lọc dữ liệu
        female_data = df[(df['subject']==subject) & (df['studytime']==level) & (df['sex']=='F')]
        male_data = df[(df['subject']==subject) & (df['studytime']==level) & (df['sex']=='M')]
        
        if len(female_data) > 0 and len(male_data) > 0:
            f_eff = female_data['efficiency'].mean()
            m_eff = male_data['efficiency'].mean()
            diff = m_eff - f_eff # Dương là Nam cao hơn, Âm là Nữ cao hơn
            pct = (diff / f_eff * 100) if f_eff != 0 else 0
            
            interaction_gender_data.append({
                'Môn': subject_name,
                'Level': f"Lv {level} ({hours}h)",
                'Hiệu suất Nữ': f_eff,
                'Hiệu suất Nam': m_eff,
                'Chênh lệch (Nam - Nữ)': diff,
                '% Chênh lệch': pct
            })

df_inter_gender = pd.DataFrame(interaction_gender_data)

print("\n1. BẢNG TƯƠNG TÁC: GIỚI TÍNH ẢNH HƯỞNG THẾ NÀO Ở TỪNG MỨC HỌC?")
# Tô màu cột Chênh lệch: Xanh (Nam > Nữ), Đỏ (Nữ > Nam)
display(df_inter_gender.style.format({
    'Hiệu suất Nữ': '{:.3f}',
    'Hiệu suất Nam': '{:.3f}',
    'Chênh lệch (Nam - Nữ)': '{:+.3f}',
    '% Chênh lệch': '{:+.1f}%'
}).background_gradient(subset=['Chênh lệch (Nam - Nữ)'], cmap='RdBu', vmin=-1, vmax=1))


# ==============================================================================
# 2️⃣ TƯƠNG TÁC: STUDYTIME × AGE (DATAFRAME)
# ==============================================================================
age_groups = [(15,15,'15'), (16,16,'16'), (17,17,'17'), (18,20,'18-20'), (21,25,'20+')]
interaction_age_data = []

for subject in ['mat', 'por']:
    subject_name = 'Toán' if subject == 'mat' else 'Văn'
    for level in [1, 2, 3, 4]:
        hours = studytime_to_hours[level]
        
        for min_age, max_age, age_label in age_groups:
            age_data = df[(df['subject']==subject) & (df['studytime']==level) & 
                          (df['age']>=min_age) & (df['age']<=max_age)]
            
            # Chỉ lấy nhóm có đủ sample (>3) để tránh nhiễu
            if len(age_data) > 3:
                eff = age_data['efficiency'].mean()
                interaction_age_data.append({
                    'Môn': subject_name,
                    'Mức học': f"Lv {level} ({hours}h)",
                    'Nhóm tuổi': age_label,
                    'Hiệu suất': eff
                })

df_inter_age = pd.DataFrame(interaction_age_data)

# Pivot bảng để tạo thành ma trận dễ nhìn (Heatmap style)
df_inter_age_pivot = df_inter_age.pivot(index=['Môn', 'Mức học'], columns='Nhóm tuổi', values='Hiệu suất')
# Sắp xếp lại cột tuổi cho đúng thứ tự
cols_order = ['15', '16', '17', '18-20', '20+']
df_inter_age_pivot = df_inter_age_pivot.reindex(columns=[c for c in cols_order if c in df_inter_age_pivot.columns])

print("\n2. MA TRẬN TƯƠNG TÁC: ĐỘ TUỔI VÀ MỨC HỌC (HEATMAP)")
# Tô màu: Càng đỏ càng hiệu quả, càng xanh càng kém (hoặc ngược lại tùy cmap)
display(df_inter_age_pivot.style.format('{:.2f}').background_gradient(cmap='YlOrRd'))


# ==============================================================================
# 3️⃣ VISUALIZATION (GIỮ NGUYÊN CODE CŨ)
# ==============================================================================
fig = plt.figure(figsize=(20, 12))
fig.suptitle('Mối Liên Hệ Tương Tác: Studytime × (Gender & Age) ảnh hưởng Hiệu Suất', 
             fontsize=16, fontweight='bold', y=0.995)
gs = fig.add_gridspec(2, 2, hspace=0.35, wspace=0.35)

# ========== BIỂU ĐỒ 1: Studytime × Gender (Toán) ==========
ax1 = fig.add_subplot(gs[0, 0])
level_labels = ['<2h', '2-5h', '5-10h', '>10h']
x_pos = np.arange(len(level_labels))

mat_f_eff_by_level = [df[(df['subject']=='mat') & (df['studytime']==i) & (df['sex']=='F')]['efficiency'].mean() 
                      for i in [1,2,3,4]]
mat_m_eff_by_level = [df[(df['subject']=='mat') & (df['studytime']==i) & (df['sex']=='M')]['efficiency'].mean() 
                      for i in [1,2,3,4]]

ax1.plot(x_pos, mat_f_eff_by_level, marker='o', linewidth=3, markersize=11, label='Nữ', 
         color='#FF6B9D', markeredgecolor='black', markeredgewidth=1.5)
ax1.plot(x_pos, mat_m_eff_by_level, marker='s', linewidth=3, markersize=11, label='Nam', 
         color='#4A90E2', markeredgecolor='black', markeredgewidth=1.5)
ax1.fill_between(x_pos, mat_f_eff_by_level, alpha=0.15, color='#FF6B9D')
ax1.fill_between(x_pos, mat_m_eff_by_level, alpha=0.15, color='#4A90E2')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(level_labels)
ax1.set_ylabel('Hiệu suất (điểm/giờ)', fontsize=10, fontweight='bold')
ax1.set_title('Toán: Studytime × Gender\n(Giới tính ảnh hưởng từng mức học?)', 
              fontsize=11, fontweight='bold', pad=10)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, linestyle='--')

# ========== BIỂU ĐỒ 2: Studytime × Gender (Văn) ==========
ax2 = fig.add_subplot(gs[0, 1])

por_f_eff_by_level = [df[(df['subject']=='por') & (df['studytime']==i) & (df['sex']=='F')]['efficiency'].mean() 
                      for i in [1,2,3,4]]
por_m_eff_by_level = [df[(df['subject']=='por') & (df['studytime']==i) & (df['sex']=='M')]['efficiency'].mean() 
                      for i in [1,2,3,4]]

ax2.plot(x_pos, por_f_eff_by_level, marker='o', linewidth=3, markersize=11, label='Nữ', 
         color='#FF6B9D', markeredgecolor='black', markeredgewidth=1.5)
ax2.plot(x_pos, por_m_eff_by_level, marker='s', linewidth=3, markersize=11, label='Nam', 
         color='#4A90E2', markeredgecolor='black', markeredgewidth=1.5)
ax2.fill_between(x_pos, por_f_eff_by_level, alpha=0.15, color='#FF6B9D')
ax2.fill_between(x_pos, por_m_eff_by_level, alpha=0.15, color='#4A90E2')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(level_labels)
ax2.set_ylabel('Hiệu suất (điểm/giờ)', fontsize=10, fontweight='bold')
ax2.set_title('Văn: Studytime × Gender\n(Giới tính ảnh hưởng từng mức học?)', 
              fontsize=11, fontweight='bold', pad=10)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, linestyle='--')

# ========== BIỂU ĐỒ 3: Studytime × Age (Toán) ==========
ax3 = fig.add_subplot(gs[1, 0])

age_group_labels = ['15', '16', '17', '18-20', '20+']
for level, label in enumerate(['Lv1:<2h', 'Lv2:2-5h', 'Lv3:5-10h', 'Lv4:>10h'], 1):
    eff_by_age = []
    for min_age, max_age, _ in age_groups:
        data = df[(df['subject']=='mat') & (df['studytime']==level) & 
                  (df['age']>=min_age) & (df['age']<=max_age)]
        if len(data) > 0:
            eff_by_age.append(data['efficiency'].mean())
        else:
            eff_by_age.append(np.nan)
    ax3.plot(age_group_labels, eff_by_age, marker='o', linewidth=2.5, markersize=9, label=label)

ax3.set_ylabel('Hiệu suất (điểm/giờ)', fontsize=10, fontweight='bold')
ax3.set_title('Toán: Age × Studytime\n(Độ tuổi ảnh hưởng từng mức học?)', 
              fontsize=11, fontweight='bold', pad=10)
ax3.legend(fontsize=9, loc='best')
ax3.grid(True, alpha=0.3, linestyle='--')

# ========== BIỂU ĐỒ 4: Studytime × Age (Văn) ==========
ax4 = fig.add_subplot(gs[1, 1])

for level, label in enumerate(['Lv1:<2h', 'Lv2:2-5h', 'Lv3:5-10h', 'Lv4:>10h'], 1):
    eff_by_age = []
    for min_age, max_age, _ in age_groups:
        data = df[(df['subject']=='por') & (df['studytime']==level) & 
                  (df['age']>=min_age) & (df['age']<=max_age)]
        if len(data) > 0:
            eff_by_age.append(data['efficiency'].mean())
        else:
            eff_by_age.append(np.nan)
    ax4.plot(age_group_labels, eff_by_age, marker='o', linewidth=2.5, markersize=9, label=label)

ax4.set_ylabel('Hiệu suất (điểm/giờ)', fontsize=10, fontweight='bold')
ax4.set_title('Văn: Age × Studytime\n(Độ tuổi ảnh hưởng từng mức học?)', 
              fontsize=11, fontweight='bold', pad=10)
ax4.legend(fontsize=9, loc='best')
ax4.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ HOÀN THÀNH: Phân tích mối liên hệ tương tác giữa các yếu tố")
print("="*80)

## NHẬN ĐỊNH TƯƠNG TÁC

### Studytime × Gender: Giới tính ảnh hưởng khác nhau ở mỗi mức học

**Level 1 (<2h)**: Nam vượt trội đáng kể (Nữ ~6.0 vs Nam ~8.5 điểm/giờ). Nam có phương pháp học tập hiệu quả ngay từ mức học thấp.

**Level 2 (2-5h)**: Khoảng cách giữa Nam-Nữ vẫn còn nhưng không quá lớn (Nữ ~4.0 vs Nam ~5.5 điểm/giờ). Nam duy trì lợi thế.

**Level 3 (5-10h)**: Cân bằng gần như hoàn toàn. Hiệu suất Nam-Nữ hội tụ ở mức ~1.1-1.2 điểm/giờ. Đây là mức tối ưu cho cả hai giới tính.

**Level 4 (>10h)**: Nữ duy trì tốt hơn (Nữ ~1.1 vs Nam ~1.0 điểm/giờ). Nam bắt đầu sụt giảm, cho thấy kém kiên trì ở mức học quá cao. Nữ thể hiện tính kiên trì vượt trội.

**Kết luận tương tác**: Nam và Nữ có chiến lược học tập khác biệt - Nam tối ưu ở mức vừa phải, Nữ thích ứng tốt hơn khi đối mặt với mức học cao hơn.

---

### Studytime × Age: Độ tuổi ảnh hưởng đến khả năng tận dụng giờ học

**Tuổi 15-17 (Nhóm trẻ)**: Tận dụng giờ học tốt nhất. Hiệu suất cao nhất ở Level 1 (~5-6 điểm/giờ) và duy trì tốt ở Level 3 (~1.2 điểm/giờ). Trí não trẻ, chưa bị kiệt sức, tiếp thu nhanh.

**Tuổi 18-19 (Nhóm vừa lưu ban)**: Hiệu quả bắt đầu suy giảm rõ rệt. Level 1 chỉ còn ~4-5 điểm/giờ, Level 3 giảm xuống ~1.0 điểm/giờ. Đây là thời kỳ nguy hiểm - có thể do kiến thức yếu từ lưu ban hoặc áp lực tâm lý.

**Tuổi 20+ (Nhóm lớn tuổi)**: Hiệu quả thấp nhất ở hầu hết mức học (0.8-1.0 điểm/giờ). Kém lợi từ mọi level - cho dù học bao nhiêu giờ cũng không cải thiện đáng kể. Sample nhỏ nhưng xu hướng rõ ràng.

**Kết luận tương tác**: Độ tuổi là yếu tố then chốt quyết định ROI của thời gian học. Cần can thiệp khẩn cấp ở tuổi 17-18 trước khi hiệu quả sụt giảm vĩnh viễn.

## NHẬN ĐỊNH CHUNG: TRẢ LỜI CÂU HỎI CHÍNH

### Câu hỏi: Mối liên hệ giữa các yếu tố nhân khẩu học (Giới tính, Tuổi tác) và studytime ảnh hưởng hiệu suất như thế nào?

---

### TỔNG HỢP TỪ Q1-Q4

**Q1 - Studytime**: Hiệu suất giảm dần (Law of Diminishing Returns). Mức tối ưu Level 3 (5-10h/tuần): G3=12.97, Hiệu suất=1.075 điểm/giờ. Vượt quá không có lợi ích thêm.

**Q2 - Gender**: Nam ROI cao hơn 32.6% (4.921 vs 3.712 điểm/giờ). Nam học ít hơn 24% nhưng đạt kết quả tương đương Nữ. Nam tối ưu ở mức vừa phải, Nữ kiên trì ở mức cao.

**Q3 - Age**: Hiệu suất giảm từ 4.5 (tuổi 15-16) xuống 3.5 (tuổi 19). Tuổi 18 là điểm gãy quan trọng. Tuổi 20+ nguy hiểm (chắc chắn lưu ban).

**Q4 - Subject**: Quy luật KHÔNG đồng nhất. Toán cần đầu tư nhiều giờ hơn, Văn bão hòa nhanh. Nam lợi thế rõ ở Toán, ở Văn chênh lệch nhỏ hơn.

---

### PHÂN TÍCH TƯƠNG TÁC

**Studytime × Gender**: Giới tính tác động khác nhau ở mỗi mức học. Level 1-2: Nam vượt trội. Level 3: Cân bằng. Level 4: Nữ duy trì, Nam sụt giảm.

**Studytime × Age**: Nhóm tuổi 15-17 tận dụng giờ học tốt nhất. Nhóm 18-19 hiệu quả giảm. Nhóm 20+ kém lợi từ mọi level.

---

### KẾT LUẬN

Các yếu tố NOT tác động độc lập mà tương tác phức tạp:
- Thời gian học: Có mức tối ưu (5-10h/tuần), vượt quá không mang lại lợi ích
- Giới tính: Nam ROI cao, Nữ kiên trì
- Tuổi tác: Giảm mạnh từ tuổi 18, đặc biệt nguy hiểm ở 20+
- Môn học: Toán vs Văn cần chiến lược khác nhau

### KHUYẾN NGHỊ

Học sinh: Học 5-10h/tuần, nam cần kỷ luật, nữ tối ưu phương pháp, can thiệp sớm cho nhóm vừa lưu ban ở nhóm tuổi 17-18.

Giáo viên: Thiết kế riêng cho Toán/Văn, hỗ trợ khác nhau theo giới tính, can thiệp sớm ở tuổi lưu ban.
